In [14]:
# !pip install joblib
# !pip install gensim
# !pip install glove-python-binary
# !pip install transformers
# !pip install datasets
# !pip install tokenizers

In [17]:
import time
import os
import psutil

def count_time(func):
    def int_time():
        start_time = time.time()
        func()
        over_time = time.time()
        total_time = over_time - start_time
        print("程序运行了%s秒" % total_time)
    return int_time

def count_info(func):
    def float_info():
        pid = os.getpid()
        p = psutil.Process(pid)
        info_start = p.memory_full_info().uss/1024
        func()
        info_end=p.memory_full_info().uss/1024
        print("程序占用了内存"+str(info_end-info_start)+"KB")
    return float_info

In [18]:
import numpy as np
import gc
import pandas as pd
import re
from joblib import Parallel, delayed
from gensim.models import FastText,Word2Vec
from glove import Glove,Corpus
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import (AutoModel,AutoModelForMaskedLM, 
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments,PreTrainedTokenizerFast,pipeline)
from datasets import Dataset
from sklearn.linear_model import Ridge
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
import warnings
warnings.filterwarnings('ignore')

from tqdm.auto import tqdm

In [19]:
from NLP_feature import NLP_feature

In [20]:
df_train = pd.read_csv('sub_train.csv')
df_test = pd.read_csv('sub_val.csv')

In [21]:
# df_test.head(3)

In [22]:
use_Toknizer=True
emb_mode = 'Word2Vec'
encode_mode = 'supervise'
text_columns_name = ['item_description']
target_column = df_train['price']
candidate_labels=None

In [23]:
nlp = NLP_feature()
# nlp.tokenizers
nlp.do_mlm = True
nlp.emb_size=100
nlp.n_clusters=20
@count_time
@count_info
def fit():
    return nlp.fit(df_train,
             text_columns_name,
             use_Toknizer,
             emb_mode,
             encode_mode,
             target_column,
             candidate_labels)
df = fit()

Fitting column: item_description tokenizer



Tokenizing column: item_description
Fitting column: item_description word2vec embedding
Fitting column: item_description encoder
程序占用了内存190264.0KB
程序运行了18.25020456314087秒


In [9]:
# for column in df.columns:
#     df_train[column] = df[column]

In [10]:
@count_time
@count_info
def trans():
    return nlp.transform(df_test)
test = trans()

Transforming column: item_description
Tokenizing column: item_description
程序占用了内存-3488.0KB
程序运行了1.3236894607543945秒


In [12]:
# df_train.head(1)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,category_name_meta_feature,item_description_meta_feature,name_meta_feature
0,39376,Size 8 Boots,2,Women/Shoes/Boots,Sonoma,24.0,0,Excellent condition! Worn maybe once or twice,18.0,19.0,16.0


In [13]:
# df_train.to_csv(f'mlm_{emb_mode}_{encode_mode}_autox_trn.csv',index=False)
# test.to_csv(f'mlm_{emb_mode}_{encode_mode}_autox_tst.csv',index=False)